In [1]:
# -*- coding: utf-8 -*-
from ekonlpy.sentiment import MPCK

import pandas as pd
import multiprocessing as mp
import pickle

import os, sys
import re

import preprocessing_mp_worker as worker

# Tidy BOK minutes text

Jupyter Lab에서의 multiprocessing 사용을 위해 main함수인 preprocessing()를 제외한 모든 함수는 preprocessing_mp_worker.py에 옮겨졌다. 

Code Attribution: 김현진T, Fininsight

### Korean stop words from here: 
https://www.ranks.nl/stopwords/korean

## Test code: 어떤 식으로 파싱되는지 확인. 

In [2]:
source_folder="./data_files/BOK_minutes/pdf2txt/"
output_file="./data_files/BOK_minutes/pkl/"

txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
txt_files = [f for f in txt_files if f[-4:] == '.txt']
txt_files.sort()

In [27]:
len(txt_files)

301

In [42]:
df = pd.DataFrame(columns=['date', 'minutes'])
df["Economic Situation"] = ""
df["Foreign Currency"] = ""
df["Financial Markets"] = ""
df["Monetary Policy"] = ""
df["Participant Views"] = ""
df["Government View"] = ""

df["Economic Situation count"] = ""
df["Foreign Currency count"] = ""
df["Financial Markets count"] = ""
df["Monetary Policy count"] = ""
df["Participant Views count"] = ""
df["Government View count"] = ""
df['ngrams'] =""

### 한 문서의 ngram의 갯수가 매우 많다. 이를 모두 각각 하나의 row로 만들면 df의 길이가 매우 커진다. 

## 1) 0번째 문서의 모든 ngram을 ,(comma)로 separate한 경우 (원본코드의 방법)

In [34]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = worker.preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for i, section_text in enumerate(section_texts[1:3]):
            print("doing #{} iteration...".format(i))
            with mp.Pool(processes=6) as p:
                mp_result = p.map(worker.text2ngram, section_text)
            for i in mp_result:
                ngrams += i

    df.loc[len(df)] = [txt_file.split('_-_')[0],
                       txt,
                       '@@@'.join(section_texts[0]),
                       '@@@'.join(section_texts[1]),
                       '@@@'.join(section_texts[2]),
                       '@@@'.join(section_texts[3]),
                       '@@@'.join(section_texts[4]),
                       '@@@'.join(section_texts[5]),
                       len(section_texts[0]),
                       len(section_texts[1]),
                       len(section_texts[2]),
                       len(section_texts[3]),
                       len(section_texts[4]),
                       len(section_texts[5]),
                       ngrams, # list 형태
                      ]

    

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
doing #0 iteration...
Section text type: <class 'list'>
doing #1 iteration...
Section text type: <class 'list'>
Wall time: 11.8 s


### Performance comparison: list vs str

df에 저장할 때 ngram들이 들어있는 list와 이를 , 로 join한 긴 str 둘 중 어떤 것이 더 나은지 확인해보았다. 

- ','.join(ngrams) 로 Concatenated string으로 저장할 시
    -  Wall time: 12s, 
    - 필수 columns memory sum: 96523, 
    - ngrams memory: 43274

- ngrams 로 list로 저장할 시 
    - Wall time: 11.8s, 
    - 필수 columns memory sum: 75353, 
    - ngrams memory: 22104
                      
승자는 list. 추후 사용성을 생각하더라도 list 구조가 나을 것으로 보인다. 

In [35]:
df.memory_usage(index=True, deep=True)

Index                           8
date                           67
minutes                     42656
Economic Situation           8288
Foreign Currency             2230
Financial Markets           30830
Monetary Policy                59
Participant Views              59
Government View                59
Economic Situation count       36
Foreign Currency count         36
Financial Markets count        36
Monetary Policy count          32
Participant Views count        32
Government View count          32
ngrams                      22104
dtype: int64

In [36]:
df[['date', 'minutes', 'Economic Situation', 'Foreign Currency','ngrams']].memory_usage(index=True, deep=True).sum()

75353

Peek inside ngram results... 

- '엔달러/NNG', '공급우위/NNG', '국제투자은행/NNG', 등을 보면 역시 eKoNLPy 답게 경제단어를 KoNLPy처럼 자르지 않고 잘 추출한 것을 볼 수 있다. 
-  '엔달러/NNG;환율/NNG;상승/NNG', '기대/NNG;원화/NNG;절상/NNG', '통화/NNG;절상/NNG', '전망/NNG;상향/NNG', 등을 보면 5-gram을 했기 때문에 token 묶음이 나오기도 하는 것을 볼 수 있다. ; 로 연결되어있다. 

In [26]:
df.ngrams

0    [경제/NNG, 경제/NNG, 비해/VV, 특별/NNG, 요인/NNG, 불구/NNG...
1                                                   []
2    [경제/NNG, 경제/NNG, 비해/VV, 특별/NNG, 요인/NNG, 불구/NNG...
3                                                   []
4    [국제/NNG, 유가/NNG, 오르/VV, 원화/NNG, 화간/NNG, 동조/NNG...
5    [위안/NNG, 절상/NNG, 원화/NNG, 절상/NNG, 가장/MAG, 추가/NN...
6    [금융/NNG, 불안사태/NNG, 발생/NNG, 가운데/NNG, 국제/NNG, 환투...
7    경제/NNG,경제/NNG,비해/VV,특별/NNG,요인/NNG,불구/NNG,원엔/NN...
Name: ngrams, dtype: object

## 2) 0번째 문서의 모든 ngram을 각각 row로 만들어 df에 담았을 경우

In [43]:
%%time

for txt_file in txt_files[:1]:
    with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
        txt = f.read()
        sections, section_texts = worker.preprocess_minutes(txt)

        print(txt_file)

        ngrams = []
        for i, section_text in enumerate(section_texts[1:3]):
            print("doing #{} iteration...".format(i))
            with mp.Pool(processes=6) as p:
                mp_result = p.map(worker.text2ngram, section_text)
                for ngram_list in mp_result:
                    for item in ngram_list:
                        df.loc[len(df)] = [txt_file.split('_-_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),
                                   item]


2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
doing #0 iteration...
doing #1 iteration...
Wall time: 25.1 s


### Performance check: 1) malstructured dataframe vs 2) ideal dataframe 

이번엔 각 ngram feature 하나하나가 한 row에 저장될 수 있도록 dataframe을 저장해보았다. 

이렇게 하면 이론적으로는 1)의 경우보다 더 이상적인 dataframe structure이다. 하지만 퍼포먼스를 확인했을 때:

- Wall time: 25.1s
- 필수 columns memory sum: 139711528
- ngrams memory: 252397

wall time도 늘어났고, 무엇보다 메모리를 10배 이상 차지한다는 것을 확인할 수 있다. 

어차피 전체 corpus에서 각 ngram의 hawkish/dovish/total 빈도수를 확인하는 것이 목표기 때문에 이를 pandas operation으로 바로 수행하지 못하고 리스트를 처리하거나 str을 처리하는데 오버헤드가 추가적으로 발생한다고 해도 메모리를 절약하는 것이 더 나은 선택일 것이다. 

In [44]:
df.memory_usage(index=True, deep=True)

Index                           20952
date                           175473
minutes                     111716064
Economic Situation           21706272
Foreign Currency              5840370
Financial Markets            80743770
Monetary Policy                154521
Participant Views              154521
Government View                154521
Economic Situation count        94284
Foreign Currency count          94284
Financial Markets count         94284
Monetary Policy count           83808
Participant Views count         83808
Government View count           83808
ngrams                         252397
dtype: int64

In [45]:
df[['date', 'minutes', 'Economic Situation', 'Foreign Currency','ngrams']].memory_usage(index=True, deep=True).sum()

139711528

In [46]:
df.ngrams

0         경제/NNG
1         경제/NNG
2          비해/VV
3         특별/NNG
4         요인/NNG
          ...   
2614     카드채/NNG
2615    프리미엄/NNG
2616      수준/NNG
2617      유지/NNG
2618      양호/NNG
Name: ngrams, Length: 2619, dtype: object

### main function에서 현재 분석에 쓰이지 않는 column은 제거

남긴 columns: date, minutes, Foreign Currency, Financial Markets

In [49]:
def preprocessing(source_folder="./data_files/BOK_minutes/pdf2txt/", output_file="./data_files/BOK_minutes/pkl/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    txt_files = [f for f in txt_files if f[-4:] == '.txt']
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df['ngrams'] =""
    

    for txt_file in txt_files :
        try :
            with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
                txt = f.read()
                sections, section_texts = worker.preprocess_minutes(txt)

                print(txt_file)

                ngrams = []
                for i, section_text in enumerate(section_texts[1:3]):
                    print("doing #{} iteration...".format(i))
                    with mp.Pool(processes=6) as p:
                        mp_result = p.map(worker.text2ngram, section_text)
                    for i in mp_result:
                        ngrams += i
                        
            df.loc[len(df)] = [
                txt_file.split('_-_')[0], # date
                txt, # minutes
                '@@@'.join(section_texts[1]), # Foreign Currency
                '@@@'.join(section_texts[2]), # Financial Markets
                ngrams, # ngrams (list)
            ]
            
        except :
            print('Error occurred at {}'.format(txt_file))
            print('Error: ', sys.exc_info()[0])
            pass
    
    df.to_pickle(output_file + 'minutes_ngram.pkl')

In [50]:
%%time

preprocessing()

2005-06-09_-_금융통화위원회 의사록(2005년도 제12차).txt
doing #0 iteration...
doing #1 iteration...
2005-06-23_-_금융통화위원회 의사록(2005년도 제13차).txt
doing #0 iteration...
doing #1 iteration...
2005-07-07_-_금융통화위원회 의사록(2005년도 제14차).txt
doing #0 iteration...
doing #1 iteration...
2005-08-11_-_금융통화위원회 의사록(2005년도 제17차).txt
doing #0 iteration...
doing #1 iteration...
2005-09-08_-_금융통화위원회 의사록(2005년도 제19차).txt
doing #0 iteration...
doing #1 iteration...
2005-09-22_-_금융통화위원회 의사록(2005년도 제20차).txt
doing #0 iteration...
doing #1 iteration...
2005-10-11_-_금융통화위원회 의사록(2005년도 제21차).txt
doing #0 iteration...
doing #1 iteration...
2005-11-10_-_금융통화위원회 의사록(2005년도 제24차).txt
doing #0 iteration...
doing #1 iteration...
2005-12-08_-_금융통화위원회 의사록(2005년도 제26차).txt
doing #0 iteration...
doing #1 iteration...
2005-12-22_-_금융통화위원회 의사록(2005년도 제27차).txt
doing #0 iteration...
doing #1 iteration...
2006-01-05_-_금융통화위원회 의사록(2006년도 제1차).txt
doing #0 iteration...
doing #1 iteration...
2006-01-12_-_금융통화위원회 의사록(2006년도 제2차).txt
doing #0 itera